<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Tutorial_for_Neural_Network_Verification_with_Marabou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for Neural Network Verification with Marabou

---

*As an example we try to verifiy the **robustness** of a classification Network trained on the MNIST dataset using the [Marabou library](https://github.com/NeuralNetworkVerification/Marabou).*

[Marabou documenation](https://neuralnetworkverification.github.io/Marabou/)

---

## Neural Network Training

In [1]:
%%capture

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/mnist_trainer.py'}

In [2]:
import mnist_trainer

First, we will train a neural network with the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset

In [3]:
import torch
from torch import nn

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, number_of_neurons):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, output_dim),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
%%capture
train_dataloader, test_dataloader = mnist_trainer.load_dataset(batch_size=64)

In [7]:
model = mnist_trainer.train_model(
    NeuralNetwork(input_dim=28*28, output_dim=10, number_of_neurons=20), 
    epochs=3, 
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    )

Epoch 1
-------------------------------


100%|██████████| 938/938 [00:10<00:00, 86.33it/s]


Test Error: 
 Accuracy: 90.5%, Avg loss: 0.329400 

Epoch 2
-------------------------------


100%|██████████| 938/938 [00:10<00:00, 86.31it/s]


Test Error: 
 Accuracy: 91.9%, Avg loss: 0.271454 

Epoch 3
-------------------------------


...

Test Error: 
 Accuracy: 92.7%, Avg loss: 0.245227 

Epoch 4
-------------------------------


100%|██████████| 938/938 [00:10<00:00, 89.46it/s]


Test Error: 
 Accuracy: 93.0%, Avg loss: 0.228384 

Epoch 5
-------------------------------


100%|██████████| 938/938 [00:10<00:00, 88.91it/s]


Test Error: 
 Accuracy: 93.5%, Avg loss: 0.209388 

Done!


## Export the neural network

## Installing Marabou

In [17]:
!git clone https://github.com/NeuralNetworkVerification/Marabou.git

Cloning into 'Marabou'...
remote: Enumerating objects: 25698, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 25698 (delta 4), reused 1 (delta 0), pack-reused 25686
Receiving objects: 100% (25698/25698), 208.52 MiB | 25.45 MiB/s, done.
Resolving deltas: 100% (18521/18521), done.


In [ ]:
from maraboupy import Marabou
